# Organize with EDA
I'm taking a look only for title, but eventually, both title and text will be used.
- For future: mind dots (handle dots without hurting sentence tokenization) in text
- Ignore comment about text for now. 

In [1]:
from freq_utils import *
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.help import upenn_tagset
from nltk.tokenize import TreebankWordTokenizer

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, RegexpParser
from nltk.corpus import stopwords, wordnet

import regex as re

from collections import Counter
import time

pd.options.display.max_colwidth = 200

In [2]:
df0 = pd.read_csv('data/True.csv')
df1 = pd.read_csv('data/Fake.csv')

In [3]:
# Drop data we don't use (learnt from eda_raw.ipynb)
df0.drop(['text','subject','date'], axis=1, inplace=True)
df1.drop(['text','subject','date'], axis=1, inplace=True)

df0.drop_duplicates()
df1.drop_duplicates()

df0 = df0[df0.title.str.split().str.len()>2]
df1 = df1[df1.title.str.split().str.len()>2]

df0['org_title'] = df0.title.copy()
df1['org_title'] = df1.title.copy()

In [4]:
# Useful function to display rows selectively
def print_sentences_with_this_string(this_string, column_to_look, df_list, df_names, 
                                     print_words=False, print_set=False, sent_token=False):
    
    n_dataFrame = len(df_list)
    
    pat = re.compile(this_string)
    
    set_list = []
    
    for i in range(n_dataFrame):
        df = df_list[i]
        df = df[df[column_to_look].str.contains(this_string, regex= True, na=False)]
        
        count = df[column_to_look].count()
        
        
        print(this_string,'in',column_to_look,'\n',df_names[i],':',count)
        
        if count==0:
            continue
        
        
        if print_set:
            df = df.sample(min(len(df),1000), random_state=20)
        else:
            df = df.sample(min(len(df),20), random_state=20)
        
        corpus_list = df[column_to_look].to_numpy()
        index_list  = df.index.to_numpy()
                
        example_df = pd.DataFrame(columns=['index','selected_text','selected_words'])
        
      
        for row in range(len(index_list)):
            
            if sent_token:
                sentences = sent_tokenize(corpus_list[row])
            
                display_text = ''
                display_word = []
            
                for sentence in sentences:
                
                    if pat.search(sentence):
                        display_text += sentence+' '
                        display_word += pat.findall(sentence)
                                        
                example_df.loc[row] = [index_list[row],display_text,display_word]
            else:
                if pat.search(corpus_list[row]):
               
                    display_text = corpus_list[row]
                    display_word = pat.findall(display_text)
                    example_df.loc[row] = [index_list[row],display_text,display_word]
                
            
            
        example_df.set_index('index')
        
        if print_set:    
            #word_set = set()
            word_set = list()
            
            lst_list = list(example_df.selected_words)
            
            for lst in lst_list:
                word_set += lst
                
            #print(word_set)
            
            word_counter = Counter(word_set)
            print(word_counter.most_common(200))
        

        if not print_words:
            example_df.drop(['selected_words'], axis=1, inplace=True)


        
        display(example_df.sample(min(len(df),20), random_state=20))
        
        if print_set:
            set_list.append(word_set)
            
    if print_set:
        return set_list

In [5]:
def replace_regex(old_exp, new_exp='_no_arg_', verbose=False):
    
    if verbose:
        print_sentences_with_this_string(old_exp, 'title', [df0,df1], ['True','Fake'], print_words=True, print_set=True)
    
    if not new_exp=='_no_arg_':
        df0.title.replace(to_replace=old_exp, value=new_exp, regex=True, inplace=True)
        df1.title.replace(to_replace=old_exp, value=new_exp, regex=True, inplace=True)
        if verbose:
            print(old_exp,'replaced to',new_exp)
            print_sentences_with_this_string(new_exp, 'title', [df0,df1], ['True','Fake'], print_words=True, print_set=True)

In [6]:
# To test
exp_list = []

# First handling itmes
- Items requiring irregular treatment.
- Process least interfering cleaning item first.



### Replace N. Korea, N.Korea to North Korea
- This word occurs frequently, so its expression better be unified.
- Do this step before removing single words.

In [7]:
# Replace N. Korea, N.Korea to North Korea
old_exp = '(?:[\s]|^)[N][\.]?[\s]?Korea'
new_exp = ' North Korea'
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

###  Remove single letter capical word
- Usually middle names, which is not necessary.
- Remove such words after treat N. Korea stuffs.

In [8]:
# Remove single letter capical word (e.g. middle name)
old_exp = '(?:[\s]|^)[A-Z][\.](?:[\s]|$)'
new_exp = ' '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

###  Abbreviation before PoS tagging
- Unify different abbreviation for a same word.
- Do this step before removing dots.
- Abbreviation items:
    - Remove a dot at the end of a Month word starts with an upper case. (Jul. -> Jul)
    - No. \<number\> to \_number_
    - UN, UK variations : U.N. and U.K. then \_u_n_ and \_u_k_ after PoS tagging
    - Rep. : Rep
    - Sept. : Sep
    - Sen. : Sen
    - Gov. : Gov
    - PM : PM, then \_prime_minister_ after PoS tagging 
    - P.M. and p.m. (post meridiem): p.m., then \_pm_ after PoS tagging (same for a.m.)
    - US, U.S, U.S. or U.S.A.: U.S., then \_u_s_ after PoS tagging

In [9]:
old_exp = '(?:[\s]|^)[N][o][\.]?[\s]?[\d]+'
new_exp = ' _number_ '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[U][\.]?[N][\.]?(?:[\s]|$)'
#new_exp = ' _u_n_ '
new_exp = ' U.N. '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[U][\.]?[K][\.]?(?:[\s]|$)'
#new_exp = ' _u_k_ '
new_exp = ' U.K. '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[U][\.]?[S][\.]?[A]?[\.]?(?:[\s]|$)'
#old_exp = '(?:[^\w]|^)[U][\.]?[S][\.]?[A]?[\.]?(?:[^\w]|$)'
#new_exp = ' _u_s_ '
new_exp = ' U.S. '
replace_regex(old_exp,new_exp, verbose=False)  
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Rr][Ee][Pp][\.]'
new_exp = ' Rep '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Ss][Ee][Pp][Tt][\.]'
new_exp = ' Sept '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Ss][Ee][Nn][\.]'
new_exp = ' Sen '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Gg][Oo][Vv][\.]'
new_exp = ' Gov '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[P][M](?:[\s]|$)'
#new_exp = ' _prime_minister_ '
new_exp = ' PM '
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Pp][\.][Mm][\.](?:[\s]|$)'
#new_exp = ' _pm_ '
new_exp = ' p.m. '
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Aa][\.][Mm][\.](?:[\s]|$)'
#new_exp = ' _am_ '
new_exp = ' a.m. '
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

# Remove a dot at the end of a word starts with an upper case.
old_exp = '(?:[\s]|^)[Jj][Aa][Nn][\.]'
new_exp = ' Jan '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Ff][Ee][Bb][\.]'
new_exp = ' Feb '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Mm][Aa][Rr][\.]'
new_exp = ' Mar '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Pp][Rr][\.]'
new_exp = ' Apr '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Jj][Uu][Nn][\.]'
new_exp = ' Jun '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Jj][Uu][Ll][\.]'
new_exp = ' Jul '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Aa][Uu][Gg][\.]'
new_exp = ' Aug '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Ss][Ee][Pp][\.]'
new_exp = ' Sep '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Oo][Cc][Tt][\.]'
new_exp = ' Oct '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Nn][Oo][Vv][\.]'
new_exp = ' Nov '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '(?:[\s]|^)[Dd][Ee][Cc][\.]'
new_exp = ' Dec '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

# Special characters, before word tokenization

### @ : Digital source or swearing
Replace any word with \@ to a tag word

In [10]:
old_exp = '[^\s]*[\@]+[^\s]*'
new_exp = ' _mytag_at_ '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

### * : Slangs
Replace any word contains \* to a tag word

In [11]:
old_exp = '[^\s]*[\*]+[^\s]*'
new_exp = ' _mytag_slang_ '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

### \/ : Date (9/11, 24/7), or clickbait (e.g. video/image)
Replace \/ to a space

In [12]:
old_exp = '[\/]'
new_exp = ' '
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

### Parentheses [] () {} : Clickbait or emphasis
~Replace to a tag with enclosed text~
Replace a parentheses to a space

In [13]:
#old_exp = '[\[\{\(\]\}\)]'
#old_exp = '[\[\{\(][\s]?[\w]+[\s]?[\]\}\)]'
#new_exp = ' '
#replace_regex(old_exp,new_exp, verbose=False)
#exp_list.append(old_exp)

### % : Percent
Replace to a word "percent"

In [14]:
old_exp = '[\%]'
new_exp = ' percent '
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

### \$ : Dollor or slangs (ignore few slang cases)
Remove

In [15]:
old_exp = '[\$]'
new_exp = ''
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

### \&: "And" or special words (e.g. "Q&A", "AT&T")

Replace to a word "and" if spaced, replace to an underbar otherwise

In [16]:
old_exp = '[\s][\&][\s]'
new_exp = ' and '
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

old_exp = '[\&]'
new_exp = '_'
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

### \# : Either special noun or number (e.g. hashtag, tv show episode, website address)
Replace to an underbar

In [17]:
old_exp = '[\#]'
new_exp = '_'
replace_regex(old_exp,new_exp, verbose=False)
exp_list.append(old_exp)

### Dots
Replace to a space

In [18]:
# dot dot, dot dot dot, or dot dot dot...
old_exp = '[\.][\.]+'
new_exp = ' '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

### Hyphen
Replace to \_, with space

In [19]:
old_exp = '[\-]'
new_exp = ' _ '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

### Quotation
Replace to a space

In [20]:
#old_exp = '(?:[\s]|^)[\'\"]'
#new_exp = ' '
#replace_regex(old_exp, new_exp, verbose=False)
#exp_list.append(old_exp)

#old_exp = '[\'\"](?:[\s]|$)'
#new_exp = ' '
#replace_regex(old_exp, new_exp, verbose=False)
#exp_list.append(old_exp)

# Number
Replace to a tag

In [21]:
old_exp = '[\d]+'
new_exp = ' _digit_ '
replace_regex(old_exp, new_exp, verbose=False)
exp_list.append(old_exp)

# Remaining special characters

In [22]:
print_sentences_with_this_string('[^\s\w]', 'title', [df0,df1], ['True','Fake'], print_words=True, print_set=True)

[^\s\w] in title 
 True : 15009
[('.', 750), ("'", 652), (',', 369), (':', 321), ('"', 14), (';', 9), ('?', 8), ('(', 4), (')', 4), ('’', 2), ('‘', 1), ('\u200b', 1)]


,index,selected_text,selected_words
890,18518,Saudi king to start Russia visit on Thursday: state news agency,[:]
694,7436,Netanyahu sees U.S. _ Israel ties 'reaching new heights' after Trump victory,"[., ., ', ']"
798,20257,"In Mexican town, women and 'muxes' take charge after massive quake","[,, ', ']"
147,1214,U.S. condemns Venezuelan elections as neither free nor fair,"[., .]"
858,21357,"British foreign secretary visits Libyan strongman, backs ceasefire","[,]"
446,20948,"Colombia's Golfo crime gang willing to surrender, president says","[', ,]"
634,5048,Watchdog presses U.S. lawmakers to probe Icahn's role with Trump,"[., ., ']"
127,1058,Twitter to label election ads after U.S. regulatory threat,"[., .]"
812,13014,"In break with decades of U.S. policy, Trump to recognize Jerusalem as Israel's capital","[., ., ,, ']"
438,2812,"MSNBC host leaving the Republican party, becoming an independent","[,]"


[^\s\w] in title 
 Fake : 21060
[('’', 639), (':', 415), ('”', 256), ('[', 253), (']', 253), ('…', 248), ('“', 242), (',', 225), ('(', 209), (')', 209), ('!', 185), ('‘', 146), ('.', 137), ('?', 84), ('–', 34), (';', 10), ('+', 2), ('″', 1), ('—', 1)]


,index,selected_text,selected_words
890,17727,BOOM! SARAH HUCKABEE SANDERS Sets Media Straight On Difference Between Trump Volunteer’s Meeting and Clinton _ DNC Paying Millions For Fake Russian Dossier,"[!, ’]"
694,13218,WATCH HILLARY SUPPORTERS Weigh In On The Second Amendment…Yikes! [Video],"[…, !, [, ]]"
798,23371,Exposed: The U.S. is an Oligarchy Ruled by Billionaires and Dictators,"[:, ., .]"
147,12447,BREAKING: PRESIDENT _ ELECT TRUMP Meets With President Obama [Video],"[:, [, ]]"
858,9242,JUST IN: “Pit Bull” Attorney For Special Counsel Robert Mueller Attended Hillary’s Election Night Party,"[:, “, ”, ’]"
446,10851,SHERIFF DAVID CLARKE Picked For Key Position In Trump Administration Aren’t We Lucky!,"[’, !]"
634,23236,"Yahoo caves in to NSA, FBI – and secretly monitors customer email accounts for U.S. Gov’t","[,, –, ., ., ’]"
127,9912,When The View’s WHOOPI GOLDBERG Told Hillary Why She Lost To Trump…Even HILLARY Was Surprised [VIDEO],"[’, …, [, ]]"
812,18537,THIRD _ RATE ACTOR Who Called His _digit_ _ Yr Old Daughter A “Rude Thoughtless Pig” Defends Liberal Pig “ISIS Kathy” Who Attacked Trump’s _digit_ _ Year Old Son,"[“, ”, “, ”, ’]"
438,19309,HYPOCRISY ON STEROIDS: Check Out HATEFUL Trump Bullies In Anti _ Bullying Ad For Kids [VIDEO],"[:, [, ]]"


[["'",
  ',',
  ':',
  ',',
  "'",
  ':',
  ',',
  '.',
  '.',
  ',',
  ':',
  '"',
  '"',
  '.',
  '.',
  ';',
  "'",
  ',',
  '.',
  '.',
  '.',
  ':',
  '.',
  '.',
  "'",
  "'",
  '.',
  '.',
  ':',
  ':',
  "'",
  "'",
  '.',
  '.',
  "'",
  "'",
  ':',
  "'",
  "'",
  '.',
  '.',
  "'",
  "'",
  '.',
  '.',
  ':',
  ',',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  ',',
  ',',
  '.',
  '.',
  "'",
  '.',
  '.',
  ':',
  ',',
  ':',
  '.',
  '.',
  '.',
  '.',
  ',',
  ':',
  "'",
  ',',
  ',',
  "'",
  "'",
  "'",
  ':',
  "'",
  ':',
  '.',
  '.',
  ',',
  ',',
  '.',
  '.',
  ',',
  ',',
  "'",
  "'",
  '.',
  '.',
  ',',
  ':',
  ',',
  '.',
  '.',
  "'",
  ',',
  ':',
  "'",
  "'",
  ',',
  ',',
  "'",
  "'",
  "'",
  '.',
  ',',
  "'",
  '.',
  '.',
  ',',
  "'",
  "'",
  ',',
  '.',
  '.',
  '.',
  '.',
  ':',
  "'",
  "'",
  ':',
  '?',
  "'",
  "'",
  ':',
  "'",
  ':',
  "'",
  "'",
  ':',
  "'",
  "'",
  ',',
  '.',
  '.',
  '.',
  '.',
  ':',
  "'",
  "'",
  ':',
  "'",

# Word tokenization and Part-of-Speech (PoS) taggins

### Tokenize and PoS tagging
Use TreebankWordTokenizer

### Abbreviation with dot
- Some abbreviations contained dots for proper PoS taggings
- Now, remove dots in the abbreviations

### Stopword and short word removal 
- Remove stop words
- Remove 1-2 letters words

In [23]:
def convert_pos(pos):
    
    tag = ''
    try:
        # get first two letters of PoS tag
        tag = pos[:2]
    except:
        tag = 'n'
    
    if tag == 'JJ':
        tag = 'a'
    elif tag == 'NN':
        tag = 'n'
    elif tag == 'RB':
        tag = 'r'
    elif tag == 'VB':
        tag = 'v'
    else:
        tag = 'n'
        
    return tag

def gen_organized_list(title):
    '''
    Apply to the title column
    '''
    
    lemmatizer = WordNetLemmatizer()
    tb_tokenizer = TreebankWordTokenizer()
    stop_words = set(stopwords.words('english'))
    clean_pat = re.compile('[\w\.]+[\']?[\w]*|[\:\;\!\?\.\'\"\,\[\{\(\]\}\)]')
    minimal_pat = re.compile('[a-z_]{3,}')
    
    # list for each row
    words_list = []
    pos_list = []
    minimal_words_list = []

    # tokenize title
    words = tb_tokenizer.tokenize(title)

    for word in words:

        # skip noisy special character
        if not bool(clean_pat.search(word)):
            continue  

        # get a list of single item
        pos = pos_tag([word])
        pos = pos[0][1]

        
        # Handle abbreviation 
        old_exp = '(?:[\s]|^)[P][M](?:[\s]|$)'
        new_exp = ' _prime_minister_ '
        word = re.sub(old_exp,new_exp,word)
        
        old_exp = '(?:[\s]|^)[Pp][\.][Mm][\.](?:[\s]|$)'
        new_exp = ' _pm_ '
        word = re.sub(old_exp,new_exp,word)

        old_exp = '(?:[\s]|^)[Aa][\.][Mm][\.](?:[\s]|$)'
        new_exp = ' _am_ '
        word = re.sub(old_exp,new_exp,word)

        old_exp = '(?:[\s]|^)[U][\.]?[S][\.]?(?:[\s]|$)'
        new_exp = ' _u_s_ '
        word = re.sub(old_exp,new_exp,word) 
        
        old_exp = '(?:[\s]|^)[U][\.]?[N][\.]?(?:[\s]|$)'
        new_exp = ' _u_n_ '
        word = re.sub(old_exp,new_exp,word) 

        old_exp = '(?:[\s]|^)[U][\.]?[K][\.]?(?:[\s]|$)'
        new_exp = ' _u_k_ '
        word = re.sub(old_exp,new_exp,word) 
      
        word = word.lower()
        
                
        # append word, pos
        pos_list.append(pos)
        words_list.append(word)

        clean_words = clean_pat.findall(word)

        # append minimal words
        for clean_word in clean_words:
            minimal_words = minimal_pat.findall(clean_word)
            for minimal_word in minimal_words:
                if not minimal_word in stop_words:
                    minimal_words_list.append(lemmatizer.lemmatize(minimal_word, convert_pos(pos)))
            
    words_list = ' '.join(words_list)
    pos_list = ' '.join(pos_list)
    minimal_words_list = ' '.join(minimal_words_list)
    
    return words_list, pos_list, minimal_words_list


temp0 = df0.title.apply(gen_organized_list)
temp1 = df1.title.apply(gen_organized_list)

In [24]:
df0['lower_title'] = df0.org_title.str.lower()
df1['lower_title'] = df1.org_title.str.lower()

df0['cleaned_words'] = temp0.apply(lambda x : x[0])
df1['cleaned_words'] = temp1.apply(lambda x : x[0])

df0['cleaned_pos'] = temp0.apply(lambda x : x[1])
df1['cleaned_pos'] = temp1.apply(lambda x : x[1])

df0['minimal_words'] = temp0.apply(lambda x : x[2])
df1['minimal_words'] = temp1.apply(lambda x : x[2])

df0.drop(['title'],axis=1,inplace=True)
df1.drop(['title'],axis=1,inplace=True)



In [36]:
display(df1)

42


,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
0,Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing,donald trump sends out embarrassing new year’s eve message; this is disturbing,donald trump sends out embarrassing new year’s eve message ; this is disturbing,NNP NN NNS IN VBG NNP NN NN NN : DT VBZ VBG,donald trump sends embarrass new year eve message disturb
1,Drunk Bragging Trump Staffer Started Russian Collusion Investigation,drunk bragging trump staffer started russian collusion investigation,drunk bragging trump staffer started russian collusion investigation,NN VBG NN NN VBN JJ NN NN,drunk brag trump staffer start russian collusion investigation
2,Sheriff David Clarke Becomes An Internet Joke For Threatening To Poke People ‘In The Eye’,sheriff david clarke becomes an internet joke for threatening to poke people ‘in the eye’,sheriff david clarke becomes an internet joke for threatening to poke people ‘in the eye’,NN NNP NN NNS DT NN NN IN VBG TO NN NNS NN DT NN,sheriff david clarke becomes internet joke threaten poke people eye
3,Trump Is So Obsessed He Even Has Obama’s Name Coded Into His Website (IMAGES),trump is so obsessed he even has obama’s name coded into his website (images),trump is so obsessed he even has obama’s name coded into his website ( images ),NN NN RB VBN PRP RB NN NN NN VBN NN PRP$ NNP ( NNS ),trump obsess even obama name cod website image
4,Pope Francis Just Called Out Donald Trump During His Christmas Speech,pope francis just called out donald trump during his christmas speech,pope francis just called out donald trump during his christmas speech,NN NN RB VBN IN NNP NN IN PRP$ NN NN,pope francis call donald trump christmas speech
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated US Sailors Well,mcpain: john mccain furious that iran treated us sailors well,mcpain : john mccain furious that iran treated _u_s_ sailors well,NN : NNP NN JJ DT NN VBN NNP NNS RB,mcpain john mccain furious iran treat _u_s_ sailor well
23477,"JUSTICE? Yahoo Settles E-mail Privacy Class-action: $4M for Lawyers, $0 for Users","justice? yahoo settles e-mail privacy class-action: $4m for lawyers, $0 for users","justice ? yahoo settles e _ mail privacy class _ action : _digit_ m for lawyers , _digit_ for users","NN . NN NNS NN NN NN NN NN NN NN : NN NN IN NNS , NN IN NNS",justice yahoo settle mail privacy class action _digit_ lawyer _digit_ user
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to Take Territorial Booty in Northern Syria,sunnistan: us and allied ‘safe zone’ plan to take territorial booty in northern syria,sunnistan : _u_s_ and allied ‘safe zone’ plan to take territorial booty in northern syria,NN : NNP CC NNP NN NN NN TO VB JJ NN IN NNP NNS,sunnistan _u_s_ allied safe zone plan take territorial booty northern syria
23479,How to Blow $700 Million: Al Jazeera America Finally Calls it Quits,how to blow $700 million: al jazeera america finally calls it quits,how to blow _digit_ million : al jazeera america finally calls it quits,WRB TO NN NN NN : NN NN NNP RB NNS PRP NNS,blow _digit_ million jazeera america finally call quits


# Confirm text processing result

In [26]:
for x in exp_list:
    #print_sentences_with_this_string(x, 'org_title', [df0,df1], ['True','Fake'], print_words=True, print_set=True)

    print(x)
    selected_row = df0.org_title.str.contains(x,regex=True)
    if len(selected_row)>0:
        display(df0[selected_row])
        
    selected_row = df1.org_title.str.contains(x,regex=True)
    if len(selected_row)>0:
        display(df1[selected_row])

(?:[\s]|^)[N][\.]?[\s]?Korea


,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
1582,"Trump slaps travel restrictions on N.Korea, Venezuela in sweeping new ban","trump slaps travel restrictions on n.korea, venezuela in sweeping new ban","trump slaps travel restrictions on north korea , venezuela in sweeping new ban","NN NNS NN NNS IN NN NNP , NNP IN VBG JJ NN",trump slap travel restriction north korea venezuela sweep new ban
1923,U.S. Treasury's Mnuchin drafting new N. Korea sanctions: Fox News,u.s. treasury's mnuchin drafting new n. korea sanctions: fox news,_u_s_ treasury 's mnuchin drafting new north korea sanctions : fox news,NNP NNP POS NN VBG JJ NN NNP NNS : NN NNS,_u_s_ treasury mnuchin draft new north korea sanction fox news
2224,"Amid nuclear standoff, frozen N.Korea debt untradeable due to sanctions","amid nuclear standoff, frozen n.korea debt untradeable due to sanctions","amid nuclear standoff , frozen north korea debt untradeable due to sanctions","IN JJ NN , NNS NN NNP NN JJ JJ TO NNS",amid nuclear standoff frozen north korea debt untradeable due sanction
10647,"U.S., backed by China, proposes tough N.Korea sanctions at U.N.","u.s., backed by china, proposes tough n.korea sanctions at u.n.","_u_s_ , backed by china , proposes tough north korea sanctions at _u_n_ .","NNP , VBD IN NNP , NNS JJ NN NNP NNS IN NN .",_u_s_ back china proposes tough north korea sanction _u_n_
12757,Russia: U.S. military drills likely aimed at provoking N.Korea,russia: u.s. military drills likely aimed at provoking n.korea,russia : _u_s_ military drills likely aimed at provoking north korea,NN : NNP JJ NNS JJ VBN IN VBG NN NNP,russia _u_s_ military drill likely aim provoke north korea
13458,Tillerson: China could do more to curb oil exports to N.Korea,tillerson: china could do more to curb oil exports to n.korea,tillerson : china could do more to curb oil exports to north korea,NN : NNP MD VB RBR TO NN NN NNS TO NN NNP,tillerson china could curb oil export north korea
13469,Kremlin says U.S. idea to cut all ties with N.Korea simplistic,kremlin says u.s. idea to cut all ties with n.korea simplistic,kremlin says _u_s_ idea to cut all ties with north korea simplistic,NNP VBZ NNP NN TO NN DT NNS IN NN NNP JJ,kremlin say _u_s_ idea cut tie north korea simplistic
13565,U.S. imposing more sanctions against N.Korea soon: White House,u.s. imposing more sanctions against n.korea soon: white house,_u_s_ imposing more sanctions against north korea soon : white house,NNP VBG RBR NNS IN NN NNP RB : NNP NNP,_u_s_ impose sanction north korea soon white house
19140,"Trump slaps travel restrictions on N.Korea, Venezuela in sweeping new ban","trump slaps travel restrictions on n.korea, venezuela in sweeping new ban","trump slaps travel restrictions on north korea , venezuela in sweeping new ban","NN NNS NN NNS IN NN NNP , NNP IN VBG JJ NN",trump slap travel restriction north korea venezuela sweep new ban
20031,Japan's Suga: government strongly protests latest N Korea missile launch,japan's suga: government strongly protests latest n korea missile launch,japan 's suga : government strongly protests latest north korea missile launch,NNP POS NN : NN RB NNS JJS NN NNP NN NN,japan suga government strongly protest late north korea missile launch


,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
9790,ACTOR JAMES WOODS DESTROYS Leftist TIME For Article Suggesting U.S. LIED About N. Korea Torturing Otto Warmbier Who Died After Returning In A Coma,actor james woods destroys leftist time for article suggesting u.s. lied about n. korea torturing otto warmbier who died after returning in a coma,actor james woods destroys leftist time for article suggesting _u_s_ lied about north korea torturing otto warmbier who died after returning in a coma,NN NN NN NN JJ NN IN NN VBG NNP NN IN NN NNP VBG NNP JJR WP VBD IN VBG IN DT NN,actor james wood destroys leftist time article suggest _u_s_ lied north korea torture otto warmbier die return coma
10187,BREAKING NEWS: GENERAL MATTIS Issues Fiery Warning To N. Korea’s Kim Jong Un…STAND DOWN Or Face “End of Its Regime…Destruction of Its People”,breaking news: general mattis issues fiery warning to n. korea’s kim jong un…stand down or face “end of its regime…destruction of its people”,breaking news : general mattis issues fiery warning to north korea’s kim jong un…stand down or face “end of its regime…destruction of its people”,NN NN : NNP NN NNS NN VBG TO NN NNP NNP RB NN RB CC NN NN IN PRP$ NN IN PRP$ NN,breaking news general mattis issue fiery warn north korea kim jong stand face end regime destruction people
10463,#FakeNewsMorningShow GMA Compares President Trump To N. Korean’s Murderous Dictator Kim Jong Un [VIDEO],#fakenewsmorningshow gma compares president trump to n. korean’s murderous dictator kim jong un [video],_fakenewsmorningshow gma compares president trump to north korean’s murderous dictator kim jong un [ video ],NN NN NNS NNP NN TO NN NNP JJ NN NNP RB NN NN NN NN,_fakenewsmorningshow gma compare president trump north korean murderous dictator kim jong video
10611,FATHER OF STUDENT Released From N. Korean Prison Slams Obama…Uses ONE WORD To Describe President Trump That Will Make Liberals Cringe [VIDEO],father of student released from n. korean prison slams obama…uses one word to describe president trump that will make liberals cringe [video],father of student released from north korean prison slams obama…uses one word to describe president trump that will make liberals cringe [ video ],NN IN NN VBN IN NN JJ NN NNS NNS CD NN TO NN NNP NN DT MD VB NNS NN NN NN NN,father student release north korean prison slam obama us one word describe president trump make liberal cringe video
10635,BREAKING NEWS: “At The Direction Of The President” 22-Yr Old American Is Released From N. Korean Prison [VIDEO],breaking news: “at the direction of the president” 22-yr old american is released from n. korean prison [video],breaking news : “at the direction of the president” _digit_ _ yr old american is released from north korean prison [ video ],NN NN : NN DT NN IN DT NN NN NN NN NN JJ NN VBN IN NN JJ NN NN NN NN,breaking news direction president _digit_ old american release north korean prison video
11109,“Crazy Fat Kid” Kim Jong Un Has HILARIOUS Rule That Prevents N. Korean Men From Copying His Look,“crazy fat kid” kim jong un has hilarious rule that prevents n. korean men from copying his look,“crazy fat kid” kim jong un has hilarious rule that prevents north korean men from copying his look,NN NN NNP NNP RB NN NN NN NN DT NNS NN JJ NN IN VBG PRP$ VB,crazy fat kid kim jong hilarious rule prevents north korean men copy look
11151,PEACE THROUGH STRENGTH: China To Cooperate With Trump On N. Korea…FLASHBACK: CHINESE DISRESPECT Weak Obama On Final Visit…Forced Him To Exit From “Ass” Of Air Force One [VIDEO],peace through strength: china to cooperate with trump on n. korea…flashback: chinese disrespect weak obama on final visit…forced him to exit from “ass” of air force one [video],peace through strength : china to cooperate with trump on north korea…flashback : chinese disrespect weak obama on final visit…forced him to exit from “ass” of air force one [ video ],NN IN NN : NNP TO NN IN NN IN NN NNP : NN NN NN NN IN JJ VBN NN TO NN IN N

(?:[\s]|^)[A-Z][\.](?:[\s]|$)


,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
1923,U.S. Treasury's Mnuchin drafting new N. Korea sanctions: Fox News,u.s. treasury's mnuchin drafting new n. korea sanctions: fox news,_u_s_ treasury 's mnuchin drafting new north korea sanctions : fox news,NNP NNP POS NN VBG JJ NN NNP NNS : NN NNS,_u_s_ treasury mnuchin draft new north korea sanction fox news
5582,Robert F. Kennedy's son announces run for Illinois governor,robert f. kennedy's son announces run for illinois governor,robert kennedy 's son announces run for illinois governor,NNP NN POS NN NNS VB IN NNP NN,robert kennedy son announces run illinois governor
7500,Former President George W. Bush does not cast vote for president,former president george w. bush does not cast vote for president,former president george bush does not cast vote for president,NN NNP NNP NNP VBZ RB NN NN IN NN,former president george bush cast vote president
8088,Ex-President George W. Bush dips toe into U.S. trade debate,ex-president george w. bush dips toe into u.s. trade debate,ex _ president george bush dips toe into _u_s_ trade debate,NN NN NNP NNP NNP NNS NN IN NNP NN NN,president george bush dip toe _u_s_ trade debate
9082,"George W. Bush ends exile, helps Republicans raise money","george w. bush ends exile, helps republicans raise money","george bush ends exile , helps republicans raise money","NNP NNP NNS NN , NNS NNPS NN NN",george bush end exile help republican raise money
10579,Wall Street's big short: President Donald J. Trump,wall street's big short: president donald j. trump,wall street 's big short : president donald trump,NNP NNP POS JJ JJ : NNP NNP NN,wall street big short president donald trump
10809,White House hopeful John Kasich hires former George W. Bush press aide,white house hopeful john kasich hires former george w. bush press aide,white house hopeful john kasich hires former george bush press aide,NNP NNP NN NNP NNP NNS JJ NNP NNP NN NN,white house hopeful john kasich hire former george bush press aide
10819,Jeb Bush gets a brotherly hand from George W. in South Carolina,jeb bush gets a brotherly hand from george w. in south carolina,jeb bush gets a brotherly hand from george in south carolina,NN NNP VBZ DT RB NN IN NNP IN NNP NNP,jeb bush get brotherly hand george south carolina
10864,George W. Bush to make first appearance for brother Jeb,george w. bush to make first appearance for brother jeb,george bush to make first appearance for brother jeb,NNP NNP TO VB RB NN IN NN NN,george bush make first appearance brother jeb
12784,UK PM May promises to uphold N. Ireland peace process as Brexit impasse ends,uk pm may promises to uphold n. ireland peace process as brexit impasse ends,_u_k_ _prime_minister_ may promises to uphold ireland peace process as brexit impasse ends,NNP NN NNP NNS TO JJ NN NN NN IN NN NN NNS,_u_k_ _prime_minister_ may promise uphold ireland peace process brexit impasse end


...

,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
3,FBI Russia probe helped by Australian diplomat tip-off: NYT,fbi russia probe helped by australian diplomat tip-off: nyt,fbi russia probe helped by australian diplomat tip _ off : nyt,NNP NN NN VBD IN JJ NN NN NN IN : NN,fbi russia probe help australian diplomat tip nyt
7,"Factbox: Trump on Twitter (Dec 29) - Approval rating, Amazon","factbox: trump on twitter (dec 29) - approval rating, amazon","factbox : trump on twitter ( dec _digit_ ) _ approval rating , amazon","NN : NN IN NN ( NN NN ) NN NN NN , NN",factbox trump twitter dec _digit_ approval rating amazon
8,Trump on Twitter (Dec 28) - Global Warming,trump on twitter (dec 28) - global warming,trump on twitter ( dec _digit_ ) _ global warming,NN IN NN ( NN NN ) NN JJ VBG,trump twitter dec _digit_ global warm
9,Alabama official to certify Senator-elect Jones today despite challenge: CNN,alabama official to certify senator-elect jones today despite challenge: cnn,alabama official to certify senator _ elect jones today despite challenge : cnn,NN NN TO NN NN NN NN NNP NN IN NN : NN,alabama official certify senator elect jones today despite challenge cnn
12,"Factbox: Trump on Twitter (Dec 28) - Vanity Fair, Hillary Clinton","factbox: trump on twitter (dec 28) - vanity fair, hillary clinton","factbox : trump on twitter ( dec _digit_ ) _ vanity fair , hillary clinton","NN : NN IN NN ( NN NN ) NN NN NN , JJ NN",factbox trump twitter dec _digit_ vanity fair hillary clinton
...,...,...,...,...,...
21347,Downfall of ex-Samsung strategy chief leaves 'salarymen' disillusioned,downfall of ex-samsung strategy chief leaves 'salarymen' disillusioned,downfall of ex _ samsung strategy chief leaves 'salarymen ' disillusioned,NN IN NN NN NN NN NN NNS NNS '' VBN,downfall samsung strategy chief leaf salarymen disillusion
21349,China says nothing will stop its long-range air force drills,china says nothing will stop its long-range air force drills,china says nothing will stop its long _ range air force drills,NNP VBZ NN MD NN PRP$ RB NN NN NN NN NNS,china say nothing stop long range air force drill
21381,Canada frets over possible huge surge in asylum-seekers: sources,canada frets over possible huge surge in asylum-seekers: sources,canada frets over possible huge surge in asylum _ seekers : sources,NNP NNS IN JJ JJ NN IN NN NN NNS : NNS,canada fret possible huge surge asylum seeker source
21385,Venezuela ex-prosecutor says she has evidence of Maduro corruption,venezuela ex-prosecutor says she has evidence of maduro corruption,venezuela ex _ prosecutor says she has evidence of maduro corruption,NNP NN NN NN VBZ PRP VBZ NN IN NN NN,venezuela prosecutor say evidence maduro corruption


,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
9,WATCH: Brand-New Pro-Trump Ad Features So Much A** Kissing It Will Make You Sick,watch: brand-new pro-trump ad features so much a** kissing it will make you sick,watch : brand _ new pro _ trump ad features so much _mytag_slang_ kissing it will make you sick,NN : NN NN NNP NNS NN NN NN NNS RB JJ NN VBG PRP MD VB PRP JJ,watch brand new pro trump feature much _mytag_slang_ kiss make sick
17,Mueller Spokesman Just F-cked Up Donald Trump’s Christmas,mueller spokesman just f-cked up donald trump’s christmas,mueller spokesman just f _ cked up donald trump’s christmas,NN NN RB NN NN NNS IN NNP NN NN,mueller spokesman cked donald trump christmas
22,Meghan McCain Tweets The Most AMAZING Response To Doug Jones’ Win In Deep-Red Alabama,meghan mccain tweets the most amazing response to doug jones’ win in deep-red alabama,meghan mccain tweets the most amazing response to doug jones’ win in deep _ red alabama,NN NN NNS DT JJS NN NN TO NN NN VB IN JJ NN JJ NN,meghan mccain tweet amazing response doug jones win deep red alabama
24,White House: It Wasn’t Sexist For Trump To Slut-Shame Sen. Kirsten Gillibrand (VIDEO),white house: it wasn’t sexist for trump to slut-shame sen. kirsten gillibrand (video),white house : it wasn’t sexist for trump to slut _ shame sen kirsten gillibrand ( video ),NNP NNP : PRP NN NN IN NN TO NN NN NN NN VB NN ( NN ),white house sexist trump slut shame sen kirsten gillibrand video
51,Trump Sends Crazy-Time Tweet To The Wrong Account After Losing His Sh*t Over World Leader’s Remarks,trump sends crazy-time tweet to the wrong account after losing his sh*t over world leader’s remarks,trump sends crazy _ time tweet to the wrong account after losing his _mytag_slang_ over world leader’s remarks,NN NNS NN NN NN NN TO DT JJ NN IN VBG PRP$ NN IN NN NN NNS,trump sends crazy time tweet wrong account lose _mytag_slang_ world leader remark
...,...,...,...,...,...
23421,"New Evidence Shows Foul Play, Cover-up by FBI and OSP in Shooting of LaVoy Finicum – DOJ Opens New Investigation","new evidence shows foul play, cover-up by fbi and osp in shooting of lavoy finicum – doj opens new investigation","new evidence shows foul play , cover _ up by fbi and osp in shooting of lavoy finicum doj opens new investigation","NNP NN NNS NN VB , NN NN RB IN NNP CC NN IN VBG IN NN NN NN NNS NNP NN",new evidence show foul play cover fbi osp shoot lavoy finicum doj open new investigation
23423,"Iraq Redux: US-led Sanctions Against Syria Are Hurting Real People, Helping Real Terrorists","iraq redux: us-led sanctions against syria are hurting real people, helping real terrorists","iraq redux : _u_s_ _ led sanctions against syria are hurting real people , helping real terrorists","NN NN : NN NN VBN NNS IN NNS NN VBG JJ NNS , VBG JJ NNS",iraq redux _u_s_ lead sanction syria hurt real people help real terrorist
23441,Wikileaks: NSA Spied on UN Secretary-General and World Leaders’ Secret Meetings,wikileaks: nsa spied on un secretary-general and world leaders’ secret meetings,wikileaks : nsa spied on _u_n_ secretary _ general and world leaders’ secret meetings,NNS : NN VBN IN NNP NNP NN NNP CC NN NN JJ NNS,wikileaks nsa spy _u_n_ secretary general world leader secret meeting
23460,Trial By YouTube: Mainstream Media Use Second-hand Oregon Account to Cast Blame on Dead Rancher,trial by youtube: mainstream media use second-hand oregon account to cast blame on dead rancher,trial by youtube : mainstream media use second _ hand oregon account to cast blame on dead rancher,NN IN NN : NN NNP NN JJ NN NN NN NN TO NN NN IN JJ NN,trial youtube mainstream medium use second hand oregon account cast blame dead rancher


[\d]+


,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
7,"Factbox: Trump on Twitter (Dec 29) - Approval rating, Amazon","factbox: trump on twitter (dec 29) - approval rating, amazon","factbox : trump on twitter ( dec _digit_ ) _ approval rating , amazon","NN : NN IN NN ( NN NN ) NN NN NN , NN",factbox trump twitter dec _digit_ approval rating amazon
8,Trump on Twitter (Dec 28) - Global Warming,trump on twitter (dec 28) - global warming,trump on twitter ( dec _digit_ ) _ global warming,NN IN NN ( NN NN ) NN JJ VBG,trump twitter dec _digit_ global warm
12,"Factbox: Trump on Twitter (Dec 28) - Vanity Fair, Hillary Clinton","factbox: trump on twitter (dec 28) - vanity fair, hillary clinton","factbox : trump on twitter ( dec _digit_ ) _ vanity fair , hillary clinton","NN : NN IN NN ( NN NN ) NN NN NN , JJ NN",factbox trump twitter dec _digit_ vanity fair hillary clinton
13,"Trump on Twitter (Dec 27) - Trump, Iraq, Syria","trump on twitter (dec 27) - trump, iraq, syria","trump on twitter ( dec _digit_ ) _ trump , iraq , syria","NN IN NN ( NN NN ) NN NN , NN , NNS",trump twitter dec _digit_ trump iraq syria
16,U.S. lawmakers question businessman at 2016 Trump Tower meeting: sources,u.s. lawmakers question businessman at 2016 trump tower meeting: sources,_u_s_ lawmakers question businessman at _digit_ trump tower meeting : sources,NNP NNS NN NN IN NN NN NN NN : NNS,_u_s_ lawmaker question businessman _digit_ trump tower meeting source
...,...,...,...,...,...
21336,Indian protests after 'godman' convicted of rape kill 29,indian protests after 'godman' convicted of rape kill 29,indian protests after 'godman ' convicted of rape kill _digit_,JJ NNS IN NN '' VBN IN NN NN NN,indian protest godman convict rape kill _digit_
21383,Brazil's Lula says party may field someone else in 2018,brazil's lula says party may field someone else in 2018,brazil 's lula says party may field someone else in _digit_,NNP POS NN VBZ NN MD NN NN RB IN NN,brazil lula say party may field someone else _digit_
21395,Poland to allocate additional $55 bllion on defense by 2032: deputy minister,poland to allocate additional $55 bllion on defense by 2032: deputy minister,poland to allocate additional _digit_ bllion on defense by _digit_ : deputy minister,NNP TO NN JJ NN NN IN NN IN NN : NN NN,poland allocate additional _digit_ bllion defense _digit_ deputy minister
21404,Exclusive: U.S. to withhold up to $290 million in Egypt aid,exclusive: u.s. to withhold up to $290 million in egypt aid,exclusive : _u_s_ to withhold up to _digit_ million in egypt aid,JJ : NNP TO NN RB TO NN CD IN NN NN,exclusive _u_s_ withhold _digit_ million egypt aid


,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words
12,Bad News For Trump — Mitch McConnell Says No To Repealing Obamacare In 2018,bad news for trump — mitch mcconnell says no to repealing obamacare in 2018,bad news for trump mitch mcconnell says no to repealing obamacare in _digit_,NN NNS IN NN NN NNP NNS DT TO VBG NN IN NN,bad news trump mitch mcconnell say repeal obamacare _digit_
23,CNN CALLS IT: A Democrat Will Represent Alabama In The Senate For The First Time In 25 Years,cnn calls it: a democrat will represent alabama in the senate for the first time in 25 years,cnn calls it : a democrat will represent alabama in the senate for the first time in _digit_ years,NN NN NN : DT NNP MD NN NN IN DT NNP IN DT RB NN IN NN NNS,cnn call democrat represent alabama senate first time _digit_ year
38,"John McCain Wanted Another 74 Twitter Followers, But His Plan Backfired Miserably (TWEETS)","john mccain wanted another 74 twitter followers, but his plan backfired miserably (tweets)","john mccain wanted another _digit_ twitter followers , but his plan backfired miserably ( tweets )","NNP NN VBD DT NN NN NNS , CC PRP$ NN VBN RB ( NN )",john mccain want another _digit_ twitter follower plan backfire miserably tweet
52,Americans Once Elected A President After He Was Accused Of Raping A 13-Year-Old Girl,americans once elected a president after he was accused of raping a 13-year-old girl,americans once elected a president after he was accused of raping a _digit_ _ year _ old girl,NNS RB VBN DT NNP IN PRP NN VBN IN VBG DT NN NN NN NN NN NN,american elect president accuse rap _digit_ year old girl
55,Sean Hannity Is Throwing A Stage-4 Temper Tantrum Over The Photo He Posed For (IMAGE),sean hannity is throwing a stage-4 temper tantrum over the photo he posed for (image),sean hannity is throwing a stage _ _digit_ temper tantrum over the photo he posed for ( image ),NN NN NN VBG DT NN NN NN NN NN IN DT NN PRP VBD IN ( NN ),sean hannity throw stage _digit_ temper tantrum photo pose image
...,...,...,...,...,...
23464,Episode #119 – SUNDAY WIRE: ‘You Know the Drill’ with guests Robert Singer and Jay Dyer,episode #119 – sunday wire: ‘you know the drill’ with guests robert singer and jay dyer,episode _ _digit_ sunday wire : ‘you know the drill’ with guests robert singer and jay dyer,NN NN NN NN NN : NN NNP DT NN IN NNS NNP NN CC NNP NN,episode _digit_ sunday wire know drill guest robert singer jay dyer
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,boiler room: as the frogs slowly boil – ep #40,boiler room : as the frogs slowly boil ep _ _digit_,NN NN : IN DT NNS RB NN NN NN NN,boiler room frog slowly boil _digit_
23477,"JUSTICE? Yahoo Settles E-mail Privacy Class-action: $4M for Lawyers, $0 for Users","justice? yahoo settles e-mail privacy class-action: $4m for lawyers, $0 for users","justice ? yahoo settles e _ mail privacy class _ action : _digit_ m for lawyers , _digit_ for users","NN . NN NNS NN NN NN NN NN NN NN : NN NN IN NNS , NN IN NNS",justice yahoo settle mail privacy class action _digit_ lawyer _digit_ user
23479,How to Blow $700 Million: Al Jazeera America Finally Calls it Quits,how to blow $700 million: al jazeera america finally calls it quits,how to blow _digit_ million : al jazeera america finally calls it quits,WRB TO NN NN NN : NN NN NNP RB NNS PRP NNS,blow _digit_ million jazeera america finally call quits


In [27]:
df0.to_csv('data/TrueOrganized.csv',index=False)
df1.to_csv('data/FakeOrganized.csv',index=False)

# Conclusion
We finished cleaning and organization.